# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            # The rows without artist information (empty strings) are skipped
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


298


In [7]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

264


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [8]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [14]:
# This cell creates a keyspace

try:
    query = "CREATE  KEYSPACE IF NOT EXISTS sparkify \
             WITH REPLICATION = {'class' : 'SimpleStrategy',\
                                 'replication_factor' : 1}"
    session.execute(query)

except Exception as e:
    print(e)

#### Set Keyspace

In [15]:
# This cell sets the keyspace to the one created above

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run. As a best practice it is recomended to run assign one table per query since JOIN and GROUP BY statements are not supported in Apache Cassandra.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

CQL Query:
```
SELECT artist, song, length FROM <table_name> WHERE sessionId = 228 and itemInSession = 4
```

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

CQL Query: song must be sorted by itemInSession
```
SELECT artist, song, firstName, lastName FROM <table_name> WHERE userid = 10, sessionid = 182
```

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

CQL Query:
```
SELECT firstName, lastName FROM <table_name> WHERE song 'All Hands Against His Own'
```

In [40]:
def create_table(table_name, fields, session, drop_first = True):
    """
    This method creates a table with the specified fields. 
    For this purpose, the session parameter must be provided.
    
    Arguments:
        table_name (str): name of the table to be created
        fields (str): column fields with data types
        session (cassandra.cluster.Session): cluster connection
        drop_first (bool): if this option is activated, the table will be dropped before creation.
        Defaults to True.
        
     Example:   
        >>> create_table("test", "(col1 int, col2 text, PRIMARY KEY(col1))", session)
    """
    
    if drop_first:
        session.execute("DROP TABLE IF EXISTS {}".format(table_name))
        print("Table {} was dropped".format(table_name))

    session.execute("CREATE TABLE IF NOT EXISTS {0} {1}".format(table_name, fields))
    print("Table {} was created".format(table_name))


def generate_fields(fields):
    """
    This method returns...
    """
    
    return "("+", ".join(fields)+")" 

In [41]:
fields = ['artist','firstName','gender','itemInSession','lastName','length',\
          'level','location','sessionId','song','userId']

generate_fields(fields)

'(artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)'

In [42]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = "SELECT artist, song, length FROM <table_name> WHERE sessionId = 228 and itemInSession = 4"

fields1 = "(artist text, \
            firstName text, \
            gender text, \
            itemInSession int, \
            lastName text, \
            length decimal, \
            level text, \
            location text, \
            sessionId int, \
            song text, \
            userId int, \
            PRIMARY KEY (sessionId, itemInSession))"

create_table(table_name = "table1", fields = fields1, session = session)

Table table1 was dropped
Table table1 was created


In [44]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
fields = ['artist','firstName','gender','itemInSession','lastName','length',\
          'level','location','sessionId','song','userId']

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # to skip header
    
    for line in csvreader:
        print(line)
        print()
        fields_str = generate_fields(fields)
        query = "INSERT INTO sparkify.table1 {} ".format(fields_str)
        values = ""
        query = query + "<ASSIGN VALUES HERE>"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        #session.execute(query, (line[#], line[#]))
        print(query)
        break

["Des'ree", 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', '139', 'You Gotta Be', '8']

INSERT INTO sparkify.table1 (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)<ASSIGN VALUES HERE>


#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


                    

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()